# Make Cube Network

In [14]:
import os
import sys
import yaml
import pickle
from subprocess import Popen

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [12]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'network_wrangler', 'examples', 'stpaul', 'pickles')
output_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'network_wrangler', 'examples', 'stpaul', 'cube')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [13]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-07-28 20:05:08, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-07-28 20:05:08, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


## Read Roadway and Transit Networks

In [16]:
roadway_net_filename = os.path.join(input_dir, 'roadway_network_with_projects.pickle')
roadway_net = pickle.load(open(roadway_net_filename, 'rb'))

In [18]:
transit_net_filename = os.path.join(input_dir, 'transit_network_with_projects.pickle')
transit_net = pickle.load(open(transit_net_filename, 'rb'))

## Make Travel Model Networks

In [19]:
model_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters,
)
model_roadway_net.roadway_standard_to_met_council_network()

2020-07-28 20:07:21, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-07-28 20:07:21, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-07-28 20:07:21, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-07-28 20:07:21, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-07-28 20:07:21, INFO: Didn't detect managed lanes in network.
2020-07-28 20:07:21, INFO: Didn't detect managed lanes in network.
2020-07-28 20:07:21, INFO: Creating calculated roadway variables.
2020-07-28 20:07:21, INFO: Creating calculated roadway variables.
2020-07-28 20:07:21, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2020-07-28 20:07:21, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:384: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-07-28 20:07:28, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-07-28 20:07:28, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-07-28 20:07:28, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp
2020-07-28 20:07:28, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:276: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-07-28 20:07:34, INFO: Finished Calculating county variable: county
2020-07-28 20:07:34, INFO: Finished Calculating county variable: county
2020-07-28 20:07:34, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-07-28 20:07:34, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-07-28 20:07:34, INFO: Finished calculating centroid connector variable: centroidconnect
2020-07-28 20:07:34, INFO: Finished calculating centroid connector variable: centroidconnect
2020-07-28 20:07:34, INFO: Calculating MPO as roadway network variable: mpo
2020-07-28 20:07:34, INFO: Calculating MPO as roadway network variable: mpo
2020-07-28 20:07:34, INFO: Finished calculating MPO variable: mpo
2020-07-28 20:07:34, INFO: Finished calculating MPO variable: mpo
2020-07-28 20:07:34, INFO: Calculating Assignment Group as network variable: assign_group
2020-07-28 20:07:34, INFO: Calculating Assignment Group as network v

## Write Networks to Disk as SHP and TXT

In [ ]:
model_roadway_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    output_link_csv = os.path.join(output_dir, 'links.csv'),
    output_node_csv = os.path.join(output_dir, 'nodes.csv'),
)

In [ ]:
model_roadway_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

In [ ]:
# standard_transit_net = StandardTransit.fromTransitNetwork(transit_net)
# standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin"))

## Make Cube .net File

In [ ]:
os.chdir(output_dir)
p = Popen("make_cube.bat")
p.communicate()